In [ ]:
from collections import defaultdict
import math
trainDict = defaultdict(list)

In [ ]:
#Sample from the training set
trainDict = {'en': ['im confused what is the real justin justinbieber jbbstratfordboy jdbstratfordboy', 'just posted a photo  pamplona spain httptcoqdaqazsb', 'alinazasoro have a very good day asieryou know that you can get a very good marc'],
             'es': ['pedazo de tarta k me e kurrao ske k buena soy en todo lo k hago jajajajaja httptcooeslfdxl', 'no hace falta que te digan que vas a morir para empezar a vivir'],
             'ca': ['joseepcoll saps q si primo arraun remo illunbe trintxerpe et fotr una pallissota quan vengui per pascu arrebet '], 
             'eu': ['anderruiz  barkauu nere', 'aitorlakun inbidi pixkat eman beharko zaizu ordun p jojojojo']}

In [ ]:
enDict = {}
esDict = {}
caDict = {}
euDict = {}
for key, value in trainDict.items():
    #get the dictionary corresponding to the language(key)
    bigramCount = 0
    if 'en' in key:
        for eachTweet in value:
            for i in range(len(eachTweet)-1):
                first = eachTweet[i]
                second = eachTweet[i+1]
                if ((not first.isspace()) and (not second.isspace())):
                    bigram = first + second
                    if bigram in enDict:
                        count = enDict[bigram]
                        count += 1
                        enDict[bigram] = count
                        bigramCount += 1
                    else:
                        enDict[bigram] = 1
                        bigramCount += 1
        enDict = {k: math.log10(v/bigramCount) for k, v in enDict.items()}
    elif 'es' in key:
        for eachTweet in value:
            for i in range(len(eachTweet)-1):
                first = eachTweet[i]
                second = eachTweet[i+1]
                if ((not first.isspace()) and (not second.isspace())):
                    bigram = first + second
                    if bigram in esDict:
                        count = esDict[bigram]
                        count += 1
                        esDict[bigram] = count
                        bigramCount += 1
                    else:
                        esDict[bigram] = 1
                        bigramCount += 1
        esDict = {k: math.log10(v/bigramCount) for k, v in esDict.items()}
    if 'eu' in key:
        for eachTweet in value:
            for i in range(len(eachTweet)-1):
                first = eachTweet[i]
                second = eachTweet[i+1]
                if ((not first.isspace()) and (not second.isspace())):
                    bigram = first + second
                    if bigram in euDict:
                        count = euDict[bigram]
                        count += 1
                        euDict[bigram] = count
                        bigramCount += 1
                    else:
                        euDict[bigram] = 1
                        bigramCount += 1
        euDict = {k: math.log10(v/bigramCount) for k, v in euDict.items()}
    if 'ca' in key:
        for eachTweet in value:
            for i in range(len(eachTweet)-1):
                first = eachTweet[i]
                second = eachTweet[i+1]
                if ((not first.isspace()) and (not second.isspace())):
                    bigram = first + second
                    if bigram in caDict:
                        count = caDict[bigram]
                        count += 1
                        caDict[bigram] = count
                        bigramCount += 1
                    else:
                        caDict[bigram] = 1
                        bigramCount += 1
        caDict = {k: math.log(v/bigramCount) for k, v in caDict.items()}


In [ ]:
print(len(enDict.keys()))